# Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.
\

In [33]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

## Step 1: Read and preprocess the dataset
---

In [35]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


### The Dataset Interface


In [36]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


In [37]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


In [38]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



In [39]:

print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')


## Step 2: Build a Most Frequent Class tagger
---


In [40]:
from collections import Counter
from collections import defaultdict
import operator


def pair_counts(sequences_A, sequences_B):

    dic = defaultdict(lambda: defaultdict(int))

    for seqA, seqB in zip(sequences_A, sequences_B):
        dic[seqA][seqB] += 1

    return dic
    

emission_counts = pair_counts([item for sub_list in data.training_set.Y for item in sub_list], [item for sub_list in data.training_set.X for item in sub_list])



In [41]:

from collections import namedtuple
import operator

FakeState = namedtuple("FakeState", "name")

class MFCTagger:

    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


word_counts = pair_counts([item for sub_list in data.training_set.X for item in sub_list],[item for sub_list in data.training_set.Y for item in sub_list])

def to_table(word_dict):
    m = defaultdict(lambda: "MISSING")
    l = {}
    for k, word in enumerate(word_dict):
        subDict = word_dict[word]
        for i in subDict:
            int(subDict[i])
        wordMax = max(subDict, key=word_dict[word].get)
        valueMax = max(subDict.values())

        m[word] = wordMax

    return m


mfc_table = to_table(word_counts) 

mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance



In [42]:
def replace_unknown(sequence):

    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):


    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [43]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

In [44]:
def accuracy(X, Y, model):

    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
    
        
        try:
            most_likely_tags = simplify_decoding(observations, model)
           
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
     
    return correct / total_predictions
    

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [45]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))



training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.01%


### Unigram Counts


In [47]:
from collections import Counter

def unigram_counts(sequences):

    dic = Counter()
    s = [item for sub_list in sequences for item in sub_list]
    for t in s:
        dic[t] += 1    
    return dic
    raise NotImplementedError

tag_unigrams = unigram_counts(data.training_set.Y)
print(tag_unigrams)


Counter({'NOUN': 220632, 'VERB': 146161, '.': 117757, 'ADP': 115808, 'DET': 109671, 'ADJ': 66754, 'ADV': 44877, 'PRON': 39383, 'CONJ': 30537, 'PRT': 23906, 'NUM': 11878, 'X': 1094})


In [48]:
from collections import Counter

def bigram_counts(sequences):

    
    dic = Counter(sequences)
    return dic
    

tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]  #got help from:  https://towardsdatascience.com/part-of-speech-tagging-with-hidden-markov-chain-models-e9fccc835c0e
biTags = [(tags[i],tags[i+1]) for i in range(0,len(tags)-1)]  #got help from: https://towardsdatascience.com/part-of-speech-tagging-with-hidden-markov-chain-models-e9fccc835c0e
tag_bigrams = bigram_counts(biTags)
print(len(tag_bigrams))


144


### Sequence Starting Counts


In [49]:
def starting_counts(sequences):

    tagList = ['CONJ', 'NOUN', 'NUM', '.','PRON', 'VERB', 'DET', 'ADP', 'ADJ', 'PRT', 'ADV', 'X']
    ee = {}

    for numa, a in enumerate(tagList):
        if a not in ee:
            cnt3 = 0
            for j in sequences:
                if a == j[0]:
                    cnt3+=1
                ee[a] = cnt3
    print(len(ee))
    return ee
    
    raise NotImplementedError

# TODO: Calculate the count of each tag starting a sequence
tag_starts = starting_counts(data.training_set.Y)


12


In [50]:
def ending_counts(sequences):

    tagList = ['CONJ', 'NOUN', 'NUM', '.','PRON', 'VERB', 'DET', 'ADP', 'ADJ', 'PRT', 'ADV', 'X']
    ff = {}

    for numa, a in enumerate(tagList):
        if a not in ff:
            cnt4 = 0
            for j in sequences:
                k=len(j)
                if a == j[k-1]:
                    cnt4+=1
                ff[a] = cnt4
    print(len(ff))
    return ff

    raise NotImplementedError


tag_ends = ending_counts(data.training_set.Y)



12


### Basic HMM Tagger


In [51]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

tagList = ['CONJ', 'NOUN', 'NUM', '.','PRON', 'VERB', 'DET', 'ADP', 'ADJ', 'PRT', 'ADV', 'X']

cUT = sum(tag_unigrams.values())
cBT = sum(tag_bigrams.values())
strT = sum(tag_starts.values())
endT = sum(tag_ends.values())

tags = [tag for i, (word, tag) in enumerate(data.stream())]

words = [word for i, (word, tag) in enumerate(data.stream())]

tag_complete_dict=pair_counts(tags,words)

create_state = {}
for tag, words_dict in tag_complete_dict.items():
    total = sum(words_dict.values())
    discTag = {word: count/total for word, count in words_dict.items()}
    tag_emissions = DiscreteDistribution(discTag)
    tag_state = State(tag_emissions, name=tag)
    create_state[tag]=tag_state
    
print(len(create_state))

for tag in create_state:
    basic_model.add_state(create_state[tag])

print("i added state")
    

startDictTag = {}
for sttag in tag_starts:
    tagVal = tag_starts[sttag]
    propStTag = tagVal/cUT
    startDictTag[sttag] = propStTag
    
for tag in create_state:
    basic_model.add_transition(basic_model.start,create_state[tag], startDictTag[tag])

print("i added starts")

endDictTag = {}
for endtag in tag_ends:
    tagVal = tag_ends[endtag]
    propEnTag = tagVal/cUT
    endDictTag[endtag] = propEnTag
    
for tag in create_state:
    basic_model.add_transition(create_state[tag], basic_model.end, endDictTag[tag])

print("i added ends")

for bittag1 in tagList:
    for bittag2 in tagList:
        callBi = (bittag1, bittag2)
        btagValue = tag_bigrams[callBi]
        countBiTag = tag_unigrams[bittag1]
        propBiTag = btagValue/countBiTag
        basic_model.add_transition(create_state[bittag1], create_state[bittag2], propBiTag)

print("i added bigrams")

    
basic_model.bake()
print("baked")


12
i added state
i added starts
i added ends
i added bigrams
baked


In [52]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))


training accuracy basic hmm model: 97.52%
testing accuracy basic hmm model: 96.13%


### Decoding Sequences with the HMM Tagger

In [53]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '.', 